In [9]:
import openai
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import time
import random
import difflib
import os
import csv

client = openai.OpenAI(
    api_key=""
)

# Đọc dữ liệu từ vnexpress_dataset.csv
df = pd.read_csv("./data/vnexpress_dataset.csv", quoting=csv.QUOTE_MINIMAL, escapechar='\\')

# Lọc tin trùng lặp từ bộ tin thật dựa trên Link
initial_count = len(df)
df = df.drop_duplicates(subset=["Link"], keep="first")
print(f"Đã loại bỏ {initial_count - len(df)} tin thật trùng lặp dựa trên Link. Còn lại {len(df)} tin thật.")

# Kiểm tra file combined dataset đã tồn tại
combined_dataset_path = "./data/vnexpress_combined_dataset.csv"
combined_links_real = set()
combined_links_fake = set()
if os.path.exists(combined_dataset_path):
    try:
        combined_df = pd.read_csv(combined_dataset_path, quoting=csv.QUOTE_MINIMAL, escapechar='\\', on_bad_lines='skip')
        # Đảm bảo có cột Label trong combined dataset
        if 'Label' not in combined_df.columns:
            combined_df['Label'] = 0  # Mặc định là tin thật
            print(f"Đã thêm cột Label vào file {combined_dataset_path}")
            # Lưu lại file với cột Label
            combined_df.to_csv(combined_dataset_path, index=False, encoding="utf-8-sig", quoting=csv.QUOTE_MINIMAL, escapechar='\\')

        # Lấy danh sách Link đã có trong combined dataset
        combined_links_real = set(combined_df[combined_df['Label'] == 0]['Link'].unique())
        combined_links_fake = set(combined_df[combined_df['Label'] == 1]['Link'].unique())
        print(f"Đã tìm thấy {len(combined_links_real)} Link tin thật và {len(combined_links_fake)} Link tin giả trong {combined_dataset_path}")
    except Exception as e:
        print(f"Lỗi khi đọc file {combined_dataset_path}: {e}")
        combined_links_real = set()
        combined_links_fake = set()

# (Tùy chọn) Lọc thêm trùng lặp dựa trên Content nếu cần
# def is_content_too_similar(content1, content2, threshold=0.95):
#     return difflib.SequenceMatcher(None, content1, content2).ratio() > threshold
#
# unique_contents = []
# unique_indices = []
# for idx, row in df.iterrows():
#     content = row["Content"]
#     if not any(is_content_too_similar(content, existing_content) for existing_content in unique_contents):
#         unique_contents.append(content)
#         unique_indices.append(idx)
# df = df.loc[unique_indices]
# print(f"Đã loại bỏ thêm {initial_count - len(unique_indices)} tin thật trùng lặp dựa trên Content. Còn lại {len(df)} tin thật.")

# Kiểm tra file tin giả đã tồn tại và lấy danh sách Link đã xử lý
processed_links = set()
fake_dataset_path = "./data/vnexpress_fake_dataset_enhance.csv"
if os.path.exists(fake_dataset_path):
    fake_df = pd.read_csv(fake_dataset_path, quoting=csv.QUOTE_MINIMAL, escapechar='\\', on_bad_lines='skip')
    processed_links = set(fake_df["Link"].unique())
    print(f"Đã tìm thấy {len(processed_links)} Link đã được xử lý trong {fake_dataset_path}")

# Lọc các tin thật chưa được xử lý và chưa có trong combined dataset
df = df[~df["Link"].isin(processed_links) & ~df["Link"].isin(combined_links_real)]
print(f"Còn lại {len(df)} tin thật chưa được xử lý để tạo tin giả")

# Hàm kiểm tra độ tương đồng giữa các nội dung
def is_too_similar(content1, content2, threshold=0.9):
    return difflib.SequenceMatcher(None, content1, content2).ratio() > threshold

# Hàm gọi API để tạo tin giả cho một batch tin thật
def generate_fake_news_batch(rows):
    try:
        # Chuẩn bị prompt cho batch
        prompt = ""
        for idx, row in enumerate(rows):
            content = row["Content"]
            prompt += f"""
            Tin thật {idx + 1}:
            [TIN THẬT]: {content}

            Hãy tạo 7 bản tin giả cho tin thật này. Mỗi bản tin giả phải:
            - Có nội dung sai sự thật nhưng hợp lý, đủ thuyết phục để có thể bị nhầm là thật.
            - Không sao chép nguyên văn tin thật, mà biến đổi câu chữ, thêm thông tin gây hiểu lầm hoặc bịa đặt.
            - Có văn phong giống báo lá cải (giật gân, hấp dẫn) hoặc bài đăng mạng xã hội (ngắn gọn, kích thích tương tác, sử dụng từ ngữ lan truyền).
            - Khác biệt rõ ràng so với tin thật và giữa các tin giả với nhau.
            - Không sử dụng các từ ngữ hoặc chi tiết vi phạm chính sách nội dung (bạo lực, phân biệt đối xử, v.v.).

            Định dạng đầu ra cho tin thật {idx + 1}:
            - Mỗi bản tin giả là một đoạn văn ngắn (2-4 câu), không đánh số, không sử dụng tiêu đề phụ.
            - Các đoạn cách nhau bằng một dòng trống.
            - Bắt đầu với dòng: "[Tin thật {idx + 1}]", theo sau là các bản tin giả.

            """
        prompt += "\nĐảm bảo các tin giả từ các tin thật khác nhau không trùng lặp nội dung."

        # Thêm cơ chế retry với exponential backoff
        max_retries = 3
        retry_delay = 1  # Thời gian chờ ban đầu (giây)
        response = []
        for attempt in range(max_retries):
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.8,
                )
                break  # Nếu thành công, thoát khỏi vòng lặp
            except Exception as e:
                if attempt < max_retries - 1:  # Nếu chưa phải lần thử cuối cùng
                    print(f"Lỗi kết nối lần {attempt+1}: {e}. Thử lại sau {retry_delay} giây...")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Tăng thời gian chờ theo cấp số nhân
                else:
                    # Nếu đã thử hết số lần cho phép, ném lại ngoại lệ
                    raise e

        fake_news = []
        current_tin_that_idx = None

        for choice in response.choices:
            # Tách nội dung theo các tin thật
            for line in choice.message.content.split("\n"):
                line = line.strip()
                if line.startswith("[Tin thật"):
                    try:
                        parts = line.split(" ")
                        if len(parts) > 2:
                            current_tin_that_idx = int(parts[2].strip("]")) - 1
                        else:
                            # If format is unexpected, skip this line
                            continue
                    except (ValueError, IndexError):
                        # If parsing fails, skip this line
                        continue
                    continue
                if line and current_tin_that_idx is not None:
                    # Check if current_tin_that_idx is valid for rows
                    if 0 <= current_tin_that_idx < len(rows):
                        row = rows[current_tin_that_idx]
                        content = row["Content"]
                        # Kiểm tra không trùng với tin thật
                        if not is_too_similar(line, content):
                            fake_news.append({
                                "Title": row["Title"],
                                "Link": row["Link"],
                                "Views": row["Views"],
                                "Comments": row["Comments"],
                                "Content": line,
                                "Label": 1
                            })
                    else:
                        # Skip if index is out of range
                        continue

        # Kiểm tra và loại bỏ tin giả trùng lặp
        unique_fake_news = []
        for news in fake_news:
            is_unique = True
            for existing_news in unique_fake_news:
                if is_too_similar(news["Content"], existing_news["Content"]):
                    is_unique = False
                    break
            if is_unique:
                unique_fake_news.append(news)

        return unique_fake_news
    except Exception as err:
        print(f"Error processing batch: {err}")
        return []

# Số lượng worker và batch size
MAX_WORKERS = 5
BATCH_SIZE = 3  # Số tin thật mỗi lần gửi API

fake_news_list = []

# Chỉ xử lý nếu còn tin thật chưa được tạo tin giả
if len(df) > 0:
    # Chia dữ liệu thành các batch
    batches = [df.iloc[i:i + BATCH_SIZE].to_dict('records') for i in range(0, len(df), BATCH_SIZE)]

    # Xử lý song song với ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Tạo dictionary của các future với tham số đầu vào
        future_to_batch = {executor.submit(generate_fake_news_batch, batch): i for i, batch in enumerate(batches)}

        # Xử lý kết quả khi hoàn thành
        for future in tqdm(concurrent.futures.as_completed(future_to_batch), total=len(future_to_batch), desc="Generating fake news"):
            try:
                results = future.result()
                # Check if results is iterable before extending
                if isinstance(results, (list, tuple)) and results:
                    fake_news_list.extend(results)
                elif results and not isinstance(results, (list, tuple)):
                    print(f"Warning: Expected iterable but got {type(results)}")
                # Giảm tải API bằng cách đợi một chút giữa các yêu cầu
                time.sleep(random.uniform(0.3, 0.7))
            except Exception as e:
                print(f"Exception occurred: {e}")

    # Lưu tin giả vào file (nối thêm vào file hiện có)
    if fake_news_list:
        fake_df = pd.DataFrame(fake_news_list)
        # Đảm bảo Label=1 cho tất cả tin giả
        if 'Label' not in fake_df.columns:
            fake_df['Label'] = 1
        if os.path.exists(fake_dataset_path):
            # Nối thêm vào file hiện có
            fake_df.to_csv(fake_dataset_path, mode='a', header=False, index=False, encoding="utf-8-sig", quoting=csv.QUOTE_MINIMAL, escapechar='\\')
        else:
            # Tạo file mới
            fake_df.to_csv(fake_dataset_path, index=False, encoding="utf-8-sig", quoting=csv.QUOTE_MINIMAL, escapechar='\\')
        print(f"✅ Đã tạo và lưu {len(fake_news_list)} tin giả mới vào {fake_dataset_path}")
else:
    print("⚠️ Không có tin thật mới nào để tạo tin giả")

# Xử lý mất cân bằng: Chọn ngẫu nhiên 2 tin giả từ mỗi tin thật
balanced_fake_news_list = []
fake_df = pd.read_csv(fake_dataset_path, quoting=csv.QUOTE_MINIMAL, escapechar='\\', on_bad_lines='skip')  # Đọc lại file để lấy tất cả tin giả
# Đảm bảo có cột Label và giá trị là 1 cho tất cả tin giả
if 'Label' not in fake_df.columns:
    fake_df['Label'] = 1
for link in fake_df["Link"].unique():
    # Bỏ qua các Link đã có tin giả trong combined dataset
    if link in combined_links_fake:
        continue

    fake_subset = fake_df[fake_df["Link"] == link].to_dict('records')
    # Chọn ngẫu nhiên 2 tin giả từ mỗi Link nếu có ít nhất 1 tin giả
    if fake_subset:  # Check if fake_subset is not empty
        selected_fakes = random.sample(fake_subset, min(2, len(fake_subset)))
        # Đảm bảo mỗi tin giả đều có Label=1
        for fake in selected_fakes:
            if 'Label' not in fake:
                fake['Label'] = 1
        balanced_fake_news_list.extend(selected_fakes)

# Tạo tập dữ liệu tin thật (sử dụng bộ tin thật đã lọc trùng lặp)
real_news_list = [
    {
        "Title": row["Title"],
        "Link": row["Link"],
        "Views": row["Views"],
        "Comments": row["Comments"],
        "Content": row["Content"],
        "Label": 0
    } for _, row in df.iterrows()
]

# Kết hợp tin thật và tin giả mới
combined_news_list = real_news_list + balanced_fake_news_list

# Nếu file combined dataset đã tồn tại, đọc nó và thêm vào các bản ghi mới
if os.path.exists(combined_dataset_path) and len(combined_news_list) > 0:
    try:
        existing_combined_df = pd.read_csv(combined_dataset_path, quoting=csv.QUOTE_MINIMAL, escapechar='\\', on_bad_lines='skip')
        # Đảm bảo có cột Label
        if 'Label' not in existing_combined_df.columns:
            existing_combined_df['Label'] = 0  # Mặc định là tin thật

        # Tạo DataFrame mới từ combined_news_list
        new_combined_df = pd.DataFrame(combined_news_list)

        # Nối DataFrame cũ và mới
        combined_df = pd.concat([existing_combined_df, new_combined_df], ignore_index=True)

        # Loại bỏ các bản ghi trùng lặp dựa trên Link và Content
        combined_df = combined_df.drop_duplicates(subset=["Link", "Content"], keep="first")

        print(f"Đã thêm {len(new_combined_df)} bản ghi mới vào file {combined_dataset_path}")
    except Exception as e:
        print(f"Lỗi khi đọc file {combined_dataset_path}: {e}")
        combined_df = pd.DataFrame(combined_news_list)
else:
    combined_df = pd.DataFrame(combined_news_list)

# Lưu vào file kết hợp nếu có dữ liệu mới
if len(combined_news_list) > 0:
    combined_df.to_csv(combined_dataset_path, index=False, encoding="utf-8-sig", quoting=csv.QUOTE_MINIMAL, escapechar='\\')

if len(combined_news_list) > 0:
    print(f"✅ Đã tạo và lưu {len(combined_news_list)} bản tin (thật + giả) mới thành công vào vnexpress_combined_dataset.csv")
else:
    print("⚠️ Không có bản tin mới nào được thêm vào vnexpress_combined_dataset.csv")

Đã loại bỏ 62 tin thật trùng lặp dựa trên Link. Còn lại 6473 tin thật.
Đã tìm thấy 4863 Link tin thật và 1314 Link tin giả trong ./data/vnexpress_combined_dataset.csv
Đã tìm thấy 1610 Link đã được xử lý trong ./data/vnexpress_fake_dataset_enhance.csv
Còn lại 0 tin thật chưa được xử lý để tạo tin giả
⚠️ Không có tin thật mới nào để tạo tin giả
Đã thêm 592 bản ghi mới vào file ./data/vnexpress_combined_dataset.csv
✅ Đã tạo và lưu 592 bản tin (thật + giả) mới thành công vào vnexpress_combined_dataset.csv
